IMPORTING NECESSARY LIBRARIES

In [1]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.python.keras.optimizers import Adam, Nadam, SGD
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import numpy as np

Using TensorFlow backend.


LOADING THE ARCHITECTURE

In [9]:
net= InceptionResNetV2(include_top=True, weights='imagenet', input_tensor=None, input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))


DATAGEN + HYPERPARAMETER

In [8]:
#PARAMETERS FOR TRAINING
DATASET_PATH = 'F://0alcohol//alcohol'
IMAGE_SIZE    = (299, 299)
NUM_CLASSES   = 2
BATCH_SIZE    = 1  # try reducing batch size or freeze more layers if your GPU runs out of memory
FREEZE_LAYERS = 40  # freeze the first this many layers for training
NUM_EPOCHS    = 5
WEIGHTS_FINAL = 'inception.hdf5'

#TO GENERATE DATA AND PREVENT CORRUPTED IMAGES
def my_gen(gen):
  while True:
    try:
      data, labels = next(gen)
      yield data, labels
    except:
      pass
# DATAGEN
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/test',
                                                  target_size=IMAGE_SIZE,
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)


Found 21936 images belonging to 2 classes.
Found 7495 images belonging to 2 classes.


TRAINING

In [ ]:
# show class indices
print('****************')
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))
print('****************')

# build our classifier model based on pre-trained InceptionResnet:
# 1. we don't include the top (fully connected) layers of InceptionResnet
# 2. we add a DropOut layer followed by a1 Dense (fully connected)
#    layer which generates softmax class score for each class
# 3. we compile the final model using an Adam optimizer, with a
#    low learning rate (since we are 'fine-tuning')

net= InceptionResNetV2(include_top=True, weights='imagenet', input_tensor=None, input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

best_weights_filepath = 'inception.hdf5'
earlyStopping=EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='auto')
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
net_final.compile(optimizer=Adam(lr=1e-3), loss="categorical_crossentropy", metrics=["accuracy"])
 
# train the model
net_final.fit_generator(my_gen  (train_batches),
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = my_gen(valid_batches),
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        callbacks=[saveBestModel, earlyStopping],
                        epochs = NUM_EPOCHS)
# save trained weights
net_final.save(WEIGHTS_FINAL)

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 284938953621169761
]


In [ ]:


from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.python.keras.optimizers import Adam, Nadam, SGD
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import numpy as np






